<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine_tuning_mixed/ask_0_mixed_pubmed_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (60) SSL certificate problem: certificate has expired
More details here: https://curl.haxx.se/docs/sslcerts.html

curl failed to verify the legitimacy of the server and therefore could not
establish a secure connection to it. To learn more about this situation and
how to fix it, please visit the web page mentioned above.


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heel pain
2,4,purple sploches around waist
3,7,goiter
4,8,ARTHRITIS


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-0.test.csv",header=None)
test.head()

,0,1
0,0,'scared' feeling
1,2,heel pain
2,3,cracking sensations in my joints
3,5,fast and slow heartbeat
4,5,increased heart rate


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_mixed.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 '.',
 'was',
 'with',
 'i',
 ',',
 'diclofenac',
 'for',
 'were',
 'on',
 'that',
 'sodium',
 'pain',
 'is',
 'by',
 'this',
 'drug',
 'as',
 'my',
 'it',
 'patients',
 'at',
 'mg',
 'or',
 'after',
 'from',
 'have',
 'not',
 'group',
 'be',
 'treatment',
 'study',
 'effects',
 'an',
 'lipitor',
 'but',
 'release',
 'p',
 'had',
 'are',
 'no',
 'all',
 'than',
 'effect',
 'two',
 'compared',
 'time',
 'day',
 'been',
 'using',
 'which',
 'groups',
 'both',
 'drugs',
 'days',
 'significantly',
 'also',
 'these',
 'has',
 'taking',
 'dose',
 'inflammatory',
 'h',
 'significant',
 'side',
 'between',
 'anti',
 'more',
 'voltaren',
 'one',
 'used',
 'results',
 'cholesterol',
 'potassium',
 'muscle',
 'when',
 'placebo',
 'efficacy',
 'we',
 '-',
 'me',
 'did',
 'ds',
 'other',
 'years',
 'showed',
 'there',
 'topical',
 'treated',
 'ml',
 'high',
 'differen

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.233150,4.638001,0.235889,15:50


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.528968,4.341028,0.263878,15:49


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.225256,4.305270,0.267255,15:59


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.179260,4.281085,0.268828,15:49


In [0]:
learn.save_encoder('bigtweet_fold0_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.792118,3.284332,0.443787,01:08


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.686527,2.750127,0.550296,01:21


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.082876,2.453780,0.601183,03:23


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.831195,2.162425,0.631953,03:28
1,1.540323,2.053022,0.654438,03:17


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.353812,1.870629,0.676923,03:28
1,1.298514,1.787432,0.692308,03:27


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.121111,1.687160,0.708876,03:41
1,1.034285,1.580914,0.724260,03:34


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.010539,1.583679,0.732544,03:45
1,0.856613,1.486736,0.743195,03:19


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.876464,1.436392,0.751479,03:24
1,0.799265,1.422956,0.757396,03:41


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.739968,1.393782,0.756213,03:39
1,0.724648,1.418446,0.762130,03:45
2,0.667171,1.365954,0.768047,03:22
3,0.542589,1.310575,0.775148,03:25


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.550950,1.349199,0.773965,03:44
1,0.584697,1.302978,0.777515,03:39
2,0.510132,1.276719,0.790533,03:44
3,0.434156,1.267025,0.790533,03:20


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.425495,1.296552,0.788166,03:44
1,0.496599,1.359811,0.770414,03:20
2,0.472411,1.295624,0.785799,03:19
3,0.401775,1.262443,0.790533,03:32


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.394652,1.290006,0.781065,03:25
1,0.457473,1.324475,0.786982,03:37
2,0.414128,1.332160,0.790533,03:26
3,0.377748,1.271340,0.790533,03:37


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.365195,1.344606,0.786982,03:39
1,0.408527,1.311408,0.783432,03:34
2,0.362889,1.333521,0.784615,03:22
3,0.331100,1.311210,0.794083,03:14


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.338311,1.316122,0.788166,03:32
1,0.377182,1.337452,0.785799,03:33
2,0.345970,1.323263,0.789349,03:24
3,0.356641,1.372479,0.790533,03:23


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.327461,1.364230,0.786982,03:48
1,0.353916,1.388503,0.788166,03:37
2,0.359650,1.436083,0.789349,03:38
3,0.293098,1.359312,0.785799,03:38


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.338932,1.339199,0.800000,03:31
1,0.315524,1.288468,0.796450,03:35


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.319725,1.383353,0.789349,03:16
1,0.330276,1.363733,0.800000,03:19


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.368929,1.376684,0.803550,03:33


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-bigtweet')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

867
867
[75, 2, 343, 787, 5, 8, 8, 8, 8, 8, 8, 8, 4, 788, 21, 22, 805, 27, 27, 541, 29, 30, 2, 30, 31, 31, 32, 34, 34, 37, 37, 491, 37, 37, 37, 37, 38, 44, 152, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 100, 100, 100, 100, 100, 53, 53, 58, 427, 71, 71, 491, 76, 76, 717, 260, 289, 289, 614, 866, 61, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 614, 614, 80, 80, 80, 80, 80, 80, 80, 80, 80, 88, 89, 137, 89, 90, 91, 92, 94, 94, 94, 94, 94, 94, 94, 94, 761, 162, 98, 423, 274, 100, 847, 337, 100, 100, 100, 100, 100, 100, 100, 291, 337, 660, 108, 108, 209, 111, 111, 111, 111, 537, 111, 289, 117, 446, 300, 300, 122, 125, 61, 630, 125, 125, 125, 125, 125, 125, 125, 125, 777, 127, 129, 622, 745, 142, 142, 144, 794, 415, 151, 151, 614, 344, 52, 158, 165, 165, 166, 166, 166, 257, 172, 700, 172, 172, 172, 172, 172, 180, 182, 182, 182, 182, 772, 289, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

579
0.6678200692041523
